In [ ]:
import os
import json
import math
from openai import OpenAI

os.environ["NO_PROXY"] = "192.168.1.10,localhost,127.0.0.1"

# 1. Configuration
# Assuming Ollama is running locally on default port
client = OpenAI(
    base_url='http://192.168.1.10:11434/v1',
    api_key='ollama',  # required, but unused by Ollama
)
MODEL_NAME = "qwen3:latest" # User asked for Qwen3, but it might not be available yet. Adjust this if you have a specific tag like 'qwen3'.

# 2. Define the Functions (The "Tools")

def calculate(expression):
    """
    Evaluates a basic mathematical expression.
    Safe for + - * / operations.
    """
    try:
        # Using eval for simplicity in this demo, but strictly limiting context is better for production
        allowed_names = {"math": math, "abs": abs, "round": round}
        code = compile(expression, "<string>", "eval")
        for name in code.co_names:
            if name not in allowed_names:
                raise NameError(f"Use of {name} is not allowed")
        result = eval(code, {"__builtins__": {}}, allowed_names)
        return json.dumps({"result": result})
    except Exception as e:
        return json.dumps({"error": str(e)})

def get_current_weather(location, unit="celsius"):
    """
    Get the current weather in a given location.
    (Mock implementation)
    """
    # In a real app, you would call an API like OpenWeatherMap here
    # Mocking data for demonstration
    weather_info = {
        "location": location,
        "temperature": "22",
        "unit": unit,
        "forecast": ["sunny", "windy"]
    }
    return json.dumps(weather_info)

# 3. Define Tool Schemas for the LLM
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Calculate the result of a mathematical expression. Useful for precise math.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate (e.g., '2 + 2', '3.5 * 40').",
                    },
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather for a specific location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        },
    },
]

# Map function names to actual Python functions
available_functions = {
    "calculate": calculate,
    "get_current_weather": get_current_weather,
}

# 4. The Agent Logic

def run_agent(user_prompt):
    messages = [{"role": "user", "content": user_prompt}]
    
    print(f"User: {user_prompt}")
    
    # First call to LLM: See if it wants to use tools
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=tools, # 模型可以使用的“工具箱”。JSON Schema 格式，告诉模型：“如果你觉得有必要，你可以调用这些函数”。
                     # 模型不会直接运行这些函数，而是返回一个“调用请求”，由你的 Python 代码去执行。
        tool_choice="auto", # "auto"（默认）：模型自动决定。它会根据用户的意图，自主判断是直接回复文字，还是调用 tools 中的某个函数。
    )
    
    response_message = response.choices[0].message
    #print(f"response_message:{response_message}")
    tool_calls = response_message.tool_calls
    
    # If the model decided to call functions
    if tool_calls:
        print(f"Agent decided to call tool(s)...")
        messages.append(response_message)  # extend conversation with assistant's reply
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)
            function_args = json.loads(tool_call.function.arguments)
            
            if function_to_call:
                print(f"  > Executing: {function_name}({function_args})")
                function_response = function_to_call(**function_args)
                
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )
        
        # Second call to LLM: Get the final answer based on tool outputs
        final_response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
        )
        return final_response.choices[0].message.content
    else:
        # No tools needed
        return response_message.content

# 5. Run interactively
if __name__ == "__main__":
    print(f"Agent initialized with model: {MODEL_NAME}")
    print("Try asking: 'What is 12345 multiplied by 6789?' or 'What's the weather in Beijing?'")
    print("-" * 50)
    
    # Example 1: Math
    q1 = "Calculate (123 + 234) * 45"
    ans1 = run_agent(q1)
    print(f"Agent: {ans1}\n")

    # Example 2: Weather
    q2 = "What is the weather like in Beijing right now?"
    ans2 = run_agent(q2)
    print(f"Agent: {ans2}\n")


Agent initialized with model: qwen3:latest
Try asking: 'What is 12345 multiplied by 6789?' or 'What's the weather in Beijing?'
--------------------------------------------------
User: Calculate (123 + 234) * 45
Agent decided to call tool(s)...
  > Executing: calculate({'expression': '(123 + 234) * 45'})
Agent: The result of the calculation is $\boxed{16065}$.

User: What is the weather like in Beijing right now?
Agent decided to call tool(s)...
  > Executing: get_current_weather({'location': 'Beijing', 'unit': 'celsius'})
Agent: The current weather in Beijing is **22°C** with a forecast of **sunny** and **windy**. Enjoy the pleasant conditions! 🌞🌬️

